<a href="https://colab.research.google.com/github/shriver42/ESAA/blob/%ED%95%84%EC%82%AC-%EA%B3%BC%EC%A0%9C/1004_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##07 문서 군집화 소개와 실습
**문서 군집화 개념**</br>
: 비슷한 텍스트 구성의 문서를 군집화(Clustering) 하는 것</br>
: 텍스트 분류와 달리 문서 군집화에서는 학습 데이터가 필요 없는 비지도 학습 기반으로 동작한다

**Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기**

In [2]:
#구글 드라이브
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 700)

#다음은 저자의 컴퓨터에서 압축 파일을 풀어놓은 디렉터리이니, 각자 디렉터리를 다시 설정합니다.
path = r'/content/drive/MyDrive/ESAA/topics'
#path로 지정한 디렉터리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

#개별 파일들의 파일명은 filename_list 리스트로 취합
#개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    #개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')

    #절대경로로 주어진 파일명을 가공. Linux에서 수행 시에는 아래 \\를 / 변경.
    #맨 마지막 .data 확장자도 제거
    filename_= file_.split('\\')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가.
    filename_list.append(filename)
    opinion_text.append(df.to_string())

#파일명 리스트와 파일 내용 리스트를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,/content/drive/MyDrive/ESAA/topics/service_holiday_inn_london,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv..."
1,/content/drive/MyDrive/ESAA/topics/service_bestwestern_hotel_sfo,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ..."
2,/content/drive/MyDrive/ESAA/topics/screen_garmin_nuvi_255W_gps,It is easy to read and when touching the screen it works great !\n0 and zoom out buttons on the 255w to the same side of the screen which makes it a bit easier .\n1 ...
3,/content/drive/MyDrive/ESAA/topics/screen_ipod_nano_8gb,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ..."
4,/content/drive/MyDrive/ESAA/topics/screen_netbook_1005ha,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...


문서를 TF—IDF 형태로 피처 벡터화

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') # Download required resource

#코드 오류 수정
def LemNormalize(text):
    """
    Lemmatizes and normalizes text
    """
    wordnet_lemmatizer = WordNetLemmatizer()
    return [wordnet_lemmatizer.lemmatize(w) for w in text.split()]

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 칼럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

[nltk_data] Downloading package wordnet to /root/nltk_data...


군집화: K-평균

In [5]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [6]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/ESAA/topics/service_holiday_inn_london,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv...",0
1,/content/drive/MyDrive/ESAA/topics/service_bestwestern_hotel_sfo,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ...",0
2,/content/drive/MyDrive/ESAA/topics/screen_garmin_nuvi_255W_gps,It is easy to read and when touching the screen it works great !\n0 and zoom out buttons on the 255w to the same side of the screen which makes it a bit easier .\n1 ...,4
3,/content/drive/MyDrive/ESAA/topics/screen_ipod_nano_8gb,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ...",2
4,/content/drive/MyDrive/ESAA/topics/screen_netbook_1005ha,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,2


군집화 결과 확인 : 군집 개수가 많아 세분화되어 군집화된 경향이 보임

In [7]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
27,/content/drive/MyDrive/ESAA/topics/food_holiday_inn_london,The room was packed to capacity with queues at the food buffets .\n0 The over zealous staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1 ...,0
25,/content/drive/MyDrive/ESAA/topics/food_swissotel_chicago,The food for our event was delicious .\n0 ...,0
37,/content/drive/MyDrive/ESAA/topics/location_holiday_inn_london,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",0
36,/content/drive/MyDrive/ESAA/topics/price_amazon_kindle,"If a case was included, as with the Kindle 1, that would have been reflected in a higher price .\n0 lower overall price, with nice leather cover .\n1 ...",0
32,/content/drive/MyDrive/ESAA/topics/price_holiday_inn_london,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",0
1,/content/drive/MyDrive/ESAA/topics/service_bestwestern_hotel_sfo,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ...",0
0,/content/drive/MyDrive/ESAA/topics/service_holiday_inn_london,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv...",0
45,/content/drive/MyDrive/ESAA/topics/service_swissotel_hotel_chicago,Mediocre room and service for a very extravagant price .\n0 ...,0


In [8]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
15,/content/drive/MyDrive/ESAA/topics/comfort_honda_accord_2008,"Drivers seat not comfortable, the car itself compared to other models of similar class .\n0 ...",1
13,/content/drive/MyDrive/ESAA/topics/comfort_toyota_camry_2007,"Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 Seats are fine, in fact of all the smaller sedans this is the most comfortable I found for the price as I am 6', 2 and 250# .\n1 Great gas mileage and comfortable on long trips ...",1
19,/content/drive/MyDrive/ESAA/topics/gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 ...,1
22,/content/drive/MyDrive/ESAA/topics/interior_honda_accord_2008,I love the new body style and the interior is a simple pleasure except for the center dash .\n0 ...,1
20,/content/drive/MyDrive/ESAA/topics/interior_toyota_camry_2007,"First of all, the interior has way too many cheap plastic parts like the cheap plastic center piece that houses the clock .\n0 3 blown struts at 30,000 miles, interior trim coming loose and rattling squeaking, stains on paint, and bug splats taking paint off, premature uneven brake wear, on 3rd windsh...",1
34,/content/drive/MyDrive/ESAA/topics/mileage_honda_accord_2008,"It's quiet, get good gas mileage and looks clean inside and out .\n0 The mileage is great, and I've had to get used to stopping less for gas .\n1 Thought gas ...",1
31,/content/drive/MyDrive/ESAA/topics/performance_honda_accord_2008,"Very happy with my 08 Accord, performance is quite adequate it has nice looks and is a great long, distance cruiser .\n0 6, 4, 3 eco engine has poor performance and gas mileage of 22 highway .\n1 Overall performance is good but comfort level is poor .\n2 ...",1
35,/content/drive/MyDrive/ESAA/topics/performance_netbook_1005ha,"The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC's to boost performance or provide better battery life depending on their immediate requirements .\n0 In Super Performance mode CPU, Z shows the bus speed to increase up to 169 .\n1 One...",1
30,/content/drive/MyDrive/ESAA/topics/quality_toyota_camry_2007,I previously owned a Toyota 4Runner which had incredible build quality and reliability .\n0 I bought the Camry because of Toyota reliability and qua...,1
5,/content/drive/MyDrive/ESAA/topics/seats_honda_accord_2008,"Front seats are very uncomfortable .\n0 No memory seats, no trip computer, can only display outside temp with trip odometer .\n1 ...",1


In [9]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,/content/drive/MyDrive/ESAA/topics/battery-life_amazon_kindle,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book one, page, at, a, time to get from Chapter 2 to Chapter 15, see how excited you are about a low battery and all the time it took to get there !\n1 ...",2
21,/content/drive/MyDrive/ESAA/topics/battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...,2
26,/content/drive/MyDrive/ESAA/topics/battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 Not to mention that as of now...",2
28,/content/drive/MyDrive/ESAA/topics/keyboard_netbook_1005ha,", I think the new keyboard rivals the great hp mini keyboards .\n0 Since the battery life difference is minimum, the only reason to upgrade would be to get the better keyboard .\n1 The keyboard is now as good as t...",2
3,/content/drive/MyDrive/ESAA/topics/screen_ipod_nano_8gb,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ...",2
4,/content/drive/MyDrive/ESAA/topics/screen_netbook_1005ha,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,2
44,/content/drive/MyDrive/ESAA/topics/size_asus_netbook_1005ha,"A few other things I'd like to point out is that you must push the micro, sized right angle end of the ac adapter until it snaps in place or the battery may not charge .\n0 The full size right shift k...",2
46,/content/drive/MyDrive/ESAA/topics/sound_ipod_nano_8gb,headphone jack i got a clear case for it and it i got a clear case for it and it like prvents me from being able to put the jack all the way in so the sound can b messsed up or i can get it in there and its playing well them go to move or something and it slides out .\n0 Picture and sound quality are excellent for this typ of devic .\n1 ...,2
40,/content/drive/MyDrive/ESAA/topics/video_ipod_nano_8gb,"I bought the 8, gig Ipod Nano that has the built, in video camera .\n0 Itunes has an on, line store, where you may purchase and download music and videos which will install onto the ipod .\n1 ...",2


In [10]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
11,/content/drive/MyDrive/ESAA/topics/bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...",3
29,/content/drive/MyDrive/ESAA/topics/free_bestwestern_hotel_sfo,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...,3
38,/content/drive/MyDrive/ESAA/topics/location_bestwestern_hotel_sfo,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",3
33,/content/drive/MyDrive/ESAA/topics/parking_bestwestern_hotel_sfo,Parking was expensive but I think this is common for San Fran .\n0 there is a fee for parking but well worth it seeing no where to park if you do have a car .\n1 ...,3
9,/content/drive/MyDrive/ESAA/topics/room_holiday_inn_london,"We arrived at 23,30 hours and they could not recommend a restaurant so we decided to go to Tesco, with very limited choices but when you are hingry you do not careNext day they rang the bell at 8,00 hours to clean the room, not being very nice being waken up so earlyEvery day they gave u...",3
8,/content/drive/MyDrive/ESAA/topics/rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , H...",3
7,/content/drive/MyDrive/ESAA/topics/rooms_swissotel_chicago,"The Swissotel is one of our favorite hotels in Chicago and the corner rooms have the most fantastic views in the city .\n0 The rooms look like they were just remodled and upgraded, there was an HD TV and a nice iHome docking station to put my iPod so I could set the alarm to wake up with my music instead of the radio .\n1 ...",3
49,/content/drive/MyDrive/ESAA/topics/staff_bestwestern_hotel_sfo,Staff are friendl...,3
47,/content/drive/MyDrive/ESAA/topics/staff_swissotel_chicago,"The staff at Swissotel were not particularly nice .\n0 Each time I waited at the counter for staff for several minutes and then was waved to the desk upon my turn with no hello or anything, or apology for waiting in line .\n1 ...",3


In [11]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
10,/content/drive/MyDrive/ESAA/topics/accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1 This functi...",4
14,/content/drive/MyDrive/ESAA/topics/buttons_amazon_kindle,"I thought it would be fitting to christen my Kindle with the Stephen King novella UR, so went to the Amazon site on my computer and clicked on the button to buy it .\n0 As soon as I'd clicked the button to confirm my order it appeared on my Kindle almost immediately !\n1 ...",4
17,/content/drive/MyDrive/ESAA/topics/directions_garmin_nuvi_255W_gps,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,4
23,/content/drive/MyDrive/ESAA/topics/display_garmin_nuvi_255W_gps,"3 quot widescreen display was a bonus .\n0 This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a jerky movement .\n1 ...",4
16,/content/drive/MyDrive/ESAA/topics/eyesight-issues_amazon_kindle,"It feels as easy to read as the K1 but doesn't seem any crisper to my eyes .\n0 the white is really GREY, and to avoid considerable eye, strain I had to refresh pages every other page .\n1 The dream has always been a portable electronic device that could hold a ton of reading material, automate subscriptions and fa...",4
24,/content/drive/MyDrive/ESAA/topics/features_windows7,"I had to uninstall anti, virus and selected other programs, some of which did not have listings in the Programs and Features Control Panel section .\n0 This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1 ...",4
12,/content/drive/MyDrive/ESAA/topics/fonts_amazon_kindle,"Being able to change the font sizes is awesome !\n0 For whatever reason, Amazon decided to make the Font on the Home Screen ...",4
39,/content/drive/MyDrive/ESAA/topics/navigation_amazon_kindle,"In fact, the entire navigation structure has been completely revised , I'm still getting used to it but it's a huge step forward .\n0 ...",4
6,/content/drive/MyDrive/ESAA/topics/satellite_garmin_nuvi_255W_gps,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ...",4
2,/content/drive/MyDrive/ESAA/topics/screen_garmin_nuvi_255W_gps,It is easy to read and when touching the screen it works great !\n0 and zoom out buttons on the 255w to the same side of the screen which makes it a bit easier .\n1 ...,4


중심 개수를 5개에서 3개로 낮춰서 군집화하기

In [12]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 소속 클러스터를 cl니ster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df ['cluster_labe'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label,cluster_labe
0,/content/drive/MyDrive/ESAA/topics/service_holiday_inn_london,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv...",0,0
45,/content/drive/MyDrive/ESAA/topics/service_swissotel_hotel_chicago,Mediocre room and service for a very extravagant price .\n0 ...,0,0
37,/content/drive/MyDrive/ESAA/topics/location_holiday_inn_london,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",0,0
36,/content/drive/MyDrive/ESAA/topics/price_amazon_kindle,"If a case was included, as with the Kindle 1, that would have been reflected in a higher price .\n0 lower overall price, with nice leather cover .\n1 ...",0,2
32,/content/drive/MyDrive/ESAA/topics/price_holiday_inn_london,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",0,0
27,/content/drive/MyDrive/ESAA/topics/food_holiday_inn_london,The room was packed to capacity with queues at the food buffets .\n0 The over zealous staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1 ...,0,0
25,/content/drive/MyDrive/ESAA/topics/food_swissotel_chicago,The food for our event was delicious .\n0 ...,0,0
1,/content/drive/MyDrive/ESAA/topics/service_bestwestern_hotel_sfo,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ...",0,0
50,/content/drive/MyDrive/ESAA/topics/transmission_toyota_camry_2007,"After slowing down, transmission has to be kicked to speed up .\n0 ...",1,1
22,/content/drive/MyDrive/ESAA/topics/interior_honda_accord_2008,I love the new body style and the interior is a simple pleasure except for the center dash .\n0 ...,1,1


**군집별 핵심 단어 추출하기**</br>
: KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심(Centroid)을 기준으로 얼마나 가깝게 위치해 있는지 clusters_centers_라는 속성으로 제공</br>
: clusters_centers_는 배열 값으로 제공되 며, 행은 개별 군집을, 열은 개별 피처를 의미함.</br>
: 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫 자 값으로 표현한 일종의 좌표 값

In [13]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape：', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape： (3, 5649)
[[0.         0.00043948 0.00099222 ... 0.00197    0.00156757 0.00156757]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00186013 0.00137598 0.         ... 0.         0.         0.        ]]


cluster_centers_ 속성값을 이용해 각 군집 별 핵심 단어를 찾기

In [14]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
    cluster_details = {}

    #cluster.centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
    #군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, :: -1]

    #개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
      #개별 군집별 정보를 담을 데이터 초기화.
      cluster_details[cluster_num] = {}
      cluster_details[cluster_num]['cluster'] = cluster_num

      #cluster_centers_.argsort()[：, :：-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함.
      top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
      top_features = [ feature_names[ind] for ind in top_feature_indexes ]

      #top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
      top_feature_values = cluster_model.cluster_centers_[cluster_num,
                                                          top_feature_indexes].tolist()

      #cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
      cluster_details[cluster_num] [ 'top_features' ] = top_features
      cluster_details[cluster_num] ['top_features_value'] = top_feature_values
      filenames = cluster_data[cluster_data['cluster_label'] == cluster_num] ['filename']
      filenames = filenames.values.tolist()

      cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [15]:
def print_cluster_details(clijster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}',format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명：', cluster_detail['filenames'][:7])
        print('============================================')

In [16]:
feature_names = tfidf_vect.get_feature_names_out() # Use get_feature_names_out() for scikit-learn version 1.0 or newer(코드 오류 수정)

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                  feature_names=feature_names, clusters_num=3, top_n_features=10)


print_cluster_details(cluster_details)

####### Cluster {0} 0
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'price', 'parking', 'clean']
Reviews 파일명： ['/content/drive/MyDrive/ESAA/topics/service_holiday_inn_london', '/content/drive/MyDrive/ESAA/topics/service_bestwestern_hotel_sfo', '/content/drive/MyDrive/ESAA/topics/food_swissotel_chicago', '/content/drive/MyDrive/ESAA/topics/food_holiday_inn_london', '/content/drive/MyDrive/ESAA/topics/price_holiday_inn_london', '/content/drive/MyDrive/ESAA/topics/price_amazon_kindle', '/content/drive/MyDrive/ESAA/topics/location_holiday_inn_london']
####### Cluster {0} 1
Top features: ['interior', 'seat', 'mileage', 'performance', 'gas', 'gas mileage', 'comfortable', 'transmission', 'car', 'quality']
Reviews 파일명： ['/content/drive/MyDrive/ESAA/topics/seats_honda_accord_2008', '/content/drive/MyDrive/ESAA/topics/comfort_toyota_camry_2007', '/content/drive/MyDrive/ESAA/topics/comfort_honda_accord_2008', '/content/drive/MyDrive/ESAA/topics/gas_mileage_to